In [1]:
from pyraf import iraf, irafpar
from pyraf.irafpar import IrafParList
import os
import math

In [2]:
def init():

    iraf.flpr()
    iraf.stsdas(_doprint=0)
    iraf.analysis(_doprint=0)
    iraf.isophote(_doprint=0)
    iraf.noao(_doprint=0)
    iraf.digiphot(_doprint=0)
    iraf.apphot(_doprint=0)
    iraf.stsdas(_doprint=0)
    iraf.analysis(_doprint=0)
    iraf.isophote(_doprint=0)
    iraf.datapars.unlearn() 
    iraf.tprint.unlearn()
    iraf.imgets.unlearn()
    iraf.mskregions.unlearn()
    iraf.fixpix.unlearn()
    iraf.controlpar.unlearn()
    iraf.geompar.unlearn()
    iraf.ellipse.unlearn()

    sma = 0.0
    maxsma = 0.0
    intens = 0.0
    tflux = 0.0
    npix = 0.0
    ellip = 0.0
    pa = 0.0
    r_ba = 0.0
    magzp = 0.0
    msky, stdev = 0.0,0.0
    newX0, newY0 = 0., 0.
    photflam = 0.0
    st_mag, ab_mag = 0., 0.
    


In [ ]:
def masking(inputdir, target, uparm, band):

    #Creating mask regions using the .reg files of the respective .fits using the mskregions task
    iraf.mskregions.setParam('regions', inputdir+target+'/'+target+'_mask.reg')
    iraf.mskregions.setParam('masks', inputdir+target+'/'+target+'_mask.fits.pl')
    iraf.mskregions.setParam('refimage', inputdir+target+'/'+target+'_'+band+'.fits')
    iraf.mskregions.setParam('append', 'no')
    iraf.mskregions.setParam('verbose', 'no')
    iraf.mskregions.saveParList(filename=uparm+'mskregions.par')
    iraf.mskregions(mode='l')
    

In [ ]:
def runEllipse(inputdir, target, x0, y0, band, magzp):
    
    iraf.unlearn(iraf.geompar)
    iraf.unlearn(iraf.controlpar)
    iraf.unlearn(iraf.magpar)
    iraf.unlearn(iraf.ellipse)
    iraf.flprcache()
    iraf.ellipse.x0=x0
    iraf.ellipse.y0=y0
    iraf.ellipse.step=0.1
    iraf.ellipse.linear='no' 
    #iraf.ellipse.wander=3
    iraf.ellipse.maxgerr=1.       #fix maxgerr=1 for all bands    
    iraf.ellipse.mag0=magzp
    iraf.ellipse.input=inputdir+target+'/'+target+'_'+band+'.fits'
    iraf.ellipse.output=inputdir+target+'/'+target+'_'+band+'_isofit.tab'  
    iraf.ellipse.region = 'no'
    iraf.ellipse.interactive = 'no'
    iraf.ellipse.verbose = 'no'
    iraf.ellipse()   

In [ ]:
def create(inputdir,target,band,newX0,newY0,sma,r_a,pa):

    iraf.display.setParam('bpdispl', 'overlay')
    iraf.display.setParam('overlay',  inputdir+target+'/'+target+'_'+band+'.fits.pl')

    #output region file with ellipse and text
    region_file = open(inputdir+target+'/'+target+'_'+band+'_isophotes.reg','w')
    region_file.write("# Region file format: DS9 version 6.2\n")
    region_file.write('#global color=green dashlist=8 3 width=1 font=\"helvetica 10 normal roman\" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1 image\n')
    region_file.write('ellipse(' + str(newX0) + ',' + str(newY0)+ ',' + str(sma0) + ','+str(round(sma0*r_a,2)) + ','+ str(90.+pa0) +  ')' + '\n')
    region_file.close()

    #constructing model & residual images
    iraf.isophote.bmodel.setParam('table', inputdir+target+'/'+target+'_'+band+'_isofit.tab')
    iraf.isophote.bmodel.setParam('output', inputdir+ target+'/'+target+'_'+band+'_model.fits')
    iraf.isophote.bmodel.setParam('parent',inputdir+target+'/'+target+'_'+band+'.fits')
    iraf.isophote.bmodel.setParam('interp','spline')
    iraf.isophote.bmodel(mode='h')
    iraf.images.imutil.imarith.setParam('operand1', inputdir+target+'/'+target+'_'+band+'.fits')
    iraf.images.imutil.imarith.setParam('op', '-')
    iraf.images.imutil.imarith.setParam('operand2', inputdir+target+'/'+target+'_'+band+'_model.fits')
    iraf.images.imutil.imarith.setParam('result', inputdir+target+'/'+target+'_'+band+'_residual.fits')
    iraf.images.imutil.imarith(mode='h')

In [9]:
#targets = ['0258+35', '1014+392', '1025+390', '1037+30', '1128+455', '1201+394', '1203+645', '1445+410']   
#1221-423 not in panstarrs rang

target_name ='1037+30'
input_dir = '/Users/orion/phd_research/CSS2/ap_phot_Panstarrs/'
uparm_dir = input_dir+'uparm/'

In [10]:
filters = ['g','r','i','z','y']
#filters = ['r','i','z','y']    #for 1014, 1201 -- not detected in g-band

In [11]:
#Enter x0,y0 manually -- wcs of Panstarrs cutouts is messed up
x0 = 372
y0 = 247

# Manual input from ANNULAR sky region --'fitsky' fails for some unknown reason
msky = 10
stdev = 88

In [12]:

for k in range(len(filters)):

    band = filters[k]

    if band == 'g' or band == 'r' or band == 'i':
        magzp = 24.6
    elif band == 'z':
        magzp = 24.3
    else:
        magzp = 23.3
        
    print band, '- band'
    init()

    for line in reversed(open(input_dir+target_name+'/'+target_name+'_'+band+'_isofit.dat').readlines()):
        print line
        line = line.strip().split()
        if line[8].strip() !='INDEF' and line[9].strip()!='INDEF':    
            sma = float(line[0].strip())
            break

    # Using msky+1sigma as the cut-off for the max sma to consider for ellipse isophote 
    # and also the one that does not have INDEF in the ellip_err and pa_err and grad and grad_err column of output table.

    # columns='SMA, intens, int_err, rms, ellip, ellip_err, 
    # pa, pa_err, grad, grad_err,rsma, mag, mag_lerr,
    # tflux_e, npix_e, X0, Y0, ndata'
  
    for line in reversed(open(input_dir+target_name+'/'+target_name+'_'+band+'_isofit.dat').readlines()):
        line = line.strip()
        line = line.split()
        if float(line[1].strip()) >= (msky+1.*stdev) and line[9].strip()!='INDEF':        #line[1] is intensity
            new_x0 = float(line[15])      #column 15,16 - we designated these columns to x0, y0 in "tdump"
            new_y0 = float(line[16])
            sma0 = float(line[0].strip())
            intens = float(line[1].strip())
            tflux = float(line[13].strip())
            npix = float(line[14].strip())
            ellip0 = float(line[4].strip())
            pa0 = float(line[6].strip())
            rmin_a = 1 - ellip0              #r-min = a(1-e)
            print 'new sma = ', sma0
            break

    print "1sigma aperture params:", target_name, new_x0, new_y0, ellip0, pa0
    print "manually entered stats: msky=", msky, 'stdev=', stdev


    #Subtracting sky background    
    tflux_bgsub = tflux - npix*msky

    #Read exposure times from header
    iraf.images.imutil.imgets(input_dir+target_name+'/'+target_name+'_'+band+'.fits', 'exptime')
    exptime = iraf.images.imutil.imgets.value

    #Panstarrs photometry relation (Tonry+2012) 
    mag = round((-2.5*math.log10(tflux_bgsub) + (2.5*math.log10(float(exptime))) + float(magzp)), 6)  
    print "Target values:"
    print "bkgd_sub_flux= ", tflux_bgsub, "exposure_time= ", exptime, "total_valid_pixels= ", npix, "mag= ", mag 

    #poisson noise (shot noise)
    n_poisson = math.sqrt(tflux)

    #error in sky
    n_sky = math.sqrt(stdev*npix)
    
    #total error
    n_tot = math.sqrt(n_poisson**2 + n_sky**2)    #add noise in quadrature
    tot_mag_err = round(2.5*math.log10(1.+n_tot/tflux), 4)
    
    print "mag_err= ", tot_mag_err

    # model & residual images and regions files
    create(input_dir,target_name,band,new_x0,new_y0,sma0,rmin_a,pa0)
    
    

g - band
     98.49736     -3.278021      2.143568      48.31357     0.3102058         INDEF      78.83247         INDEF    -0.6197719         INDEF      3.150331          24.6         INDEF      2445214.       21037      111.1308      119.4685         508

     89.54305      3.659012      2.272358      48.84248     0.3102058         INDEF      78.83247         INDEF    -0.7747148     0.3488667      3.076153      23.19159      1.053488      2439848.       17375      111.1308      119.4685         462

new sma =  41.77248
1sigma aperture params: 1037+30 108.5671 124.336 0.3040258 75.83113
manually entered stats: msky= 10 stdev= 88
Target values:
bkgd_sub_flux=  2037979.0 exposure_time=  792. total_valid_pixels=  3811.0 mag=  16.073814
mag_err=  0.0008
r - band
     89.54305      4.732398      3.054208      65.93128     0.2988118    0.06794016     -62.30427      7.723671    -0.9941189         INDEF      3.076153       22.9123      1.125599      5625652.       17669      117.9139      122